<a href="https://colab.research.google.com/github/hiyori99/gruff-pycaret/blob/main/PyCaret_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pycaret 

     |████████████████████████████████| 288 kB 5.2 MB/s 
     |████████████████████████████████| 167 kB 51.7 MB/s 
     |████████████████████████████████| 1.3 MB 41.7 MB/s 
     |████████████████████████████████| 261 kB 56.5 MB/s 
     |████████████████████████████████| 1.7 MB 49.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.5 MB 15.2 MB/s 
     |████████████████████████████████| 113 kB 40.2 MB/s 
     |████████████████████████████████| 2.0 MB 69.1 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 6.8 MB 48.3 MB/s 
     |████████████████████████████████| 675 kB 47.4 MB/s 
     |████████████████████████████████| 10.1 MB 24.4 MB/s 
     |████████████████████████████████| 3.1 MB 73.1 MB/s 
     |████████████████

In [2]:
import os
import random

import numpy as np
import pandas as pd

In [3]:
def random_seed_initialize(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [4]:
random_seed_initialize()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
train_data = pd.read_csv("drive/My Drive/gruff/train.csv")
test_data = pd.read_csv("drive/My Drive/gruff/test.csv")
submission_data = pd.read_csv("drive/My Drive/gruff/gender_submission.csv")

Mounted at /content/drive


In [7]:
from pycaret.classification import *

In [8]:
exp = setup(data=train_data, target='Survived', 
            ignore_features = ['PassengerId', 'Name'], session_id=42) 

,Description,Value
0,session_id,42
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(891, 12)"
5,Missing Values,True
6,Numeric Features,2
7,Categorical Features,7
8,Ordinal Features,False
9,High Cardinality Features,False


In [12]:
fold_start_number = 2
fold_end_number = 30 + 1
except_count = 0

for i in range(fold_start_number, fold_end_number, 1):
    try:
        blend_models(fold=i, verbose=False)
    except:
        except_count += 1

save_experiment('TitanicBlendModelsExperiment')
experiment = load_experiment('TitanicBlendModelsExperiment')

NameError: ignored

In [ ]:
accuracy = []
best_fold_number = 0
best_accuracy = 0
best_model = None

for i in range(fold_end_number - fold_start_number - except_count, 0, -1):
    fold_num = len(experiment[-i*2+1]['Accuracy']) - 2
    accuracy_mean = experiment[-i*2+1]['Accuracy']['Mean']
    model = experiment[-i*2]

    if best_accuracy < accuracy_mean:
        best_fold_number = fold_num
        best_accuracy = accuracy_mean
        best_model = model

    accuracy.append([fold_num, accuracy_mean])

print('best fold number:' + str(best_fold_number))
print('best accuracy:' + str(best_accuracy))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 7))
plt.xticks(np.arange(2, 30 + 1, 1))
plt.yticks(np.arange(0.82, 0.86, 0.001))
sns.lineplot(x="fold number", y="accuracy(mean)", data=pd.DataFrame(accuracy, columns=['fold number', 'accuracy(mean)']))

In [ ]:
predictions = predict_model(best_model, data=test_data)
predictions.head()

In [ ]:
submission_data['Survived'] = round(predictions['Label']).astype(int)
submission_data.to_csv('submission.csv',index=False)
submission_data.head()